## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

In [18]:
# Cell 2: 必要なライブラリのインポート
# このセルでは、プログラム全体で必要となるPythonライブラリをすべてインポートします。
# - os: オペレーティングシステムが提供する機能（環境変数など）にアクセスするために使用します。
# - json: JSON形式のデータをエンコードおよびデコードするために使用します。
# - dotenv: .envファイルから環境変数を読み込むために使用します。
# - openai: OpenAIのAPIにアクセスするためのクライアントライブラリです。
# - anthropic: AnthropicのAPIにアクセスするためのクライアントライブラリです。
# - IPython.display: Jupyterノートブック内でMarkdownなどのリッチなコンテンツを表示するために使用します。
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [19]:
# Cell 3: 環境変数の読み込み
# プログラムが外部サービスと連携するために必要なAPIキーなどの秘密情報を、`.env`ファイルから読み込みます。
# `override=True`を設定することで、既に設定されている環境変数も`.env`ファイルの内容で上書きします。
# これにより、APIキーをコードに直接記述するリスクを避け、セキュリティを向上させます。
load_dotenv(override=True)

True

In [62]:
# Cell 4: ユーティリティ関数群の定義
# このセルでは、LLMの比較と評価プロセス全体で使用されるすべての補助関数を定義します。
# 各関数は特定の役割を持ち、コードのモジュール化と再利用性を高めます。

def print_separator(title=""):
    """
    視覚的な区切り線とタイトルを表示する関数です。
    Jupyterノートブックの出力において、異なるセクションの開始を示すのに役立ちます。
    例: print_separator("API Key Check")
    """
    print("\n" + "="*60)
    if title:
        print(f" {title}")
        print("="*60)

def check_api_keys():
    """
    プログラムで使用される各LLMサービスのAPIキーが環境変数に設定されているかを確認する関数です。
    キーの存在と、そのキーがどのLLMサービスに属するかをユーザーに通知します。
    設定されていないキーについては警告を表示し、どのLLMがスキップされるかを明確にします。
    """
    print_separator("API Key Check")
    
    openai_api_key = os.getenv('OPENAI_API_KEY')
    anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
    google_api_key = os.getenv('GOOGLE_API_KEY')
    deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
    groq_api_key = os.getenv('GROQ_API_KEY')

    if openai_api_key:
        print(f"✅ OpenAI API Key exists and begins {openai_api_key[:8]}")
    else:
        print("❌ OpenAI API Key not set")
        
    if anthropic_api_key:
        print(f"✅ Anthropic API Key exists and begins {anthropic_api_key[:7]}")
    else:
        print("⚠️ Anthropic API Key not set (and this is optional)")

    if google_api_key:
        print(f"✅ Google API Key exists and begins {google_api_key[:2]}")
    else:
        print("⚠️ Google API Key not set (and this is optional)")

    if deepseek_api_key:
        print(f"✅ DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
    else:
        print("⚠️ DeepSeek API Key not set (and this is optional)")

    if groq_api_key:
        print(f"✅ Groq API Key exists and begins {groq_api_key[:4]}")
    else:
        print("⚠️ Groq API Key not set (and this is optional)")

def generate_question():
    """
    日本の2030年、2040年、2050年など将来を見越した、日本の政策的課題を「AIを活用すること」で解決できそうなテーマをGPT-4o-miniに生成させる関数です。
    質問は日本語で生成され、評価の基礎となります。
    API呼び出しに失敗した場合は、デフォルトの質問（日本語）を返します。
    """
    print_separator("Generating Question")
    
    request = "日本の内閣官房長官として、首相に提案すべく、日本の2030年、2040年、2050年など将来を見越した国策の検討が必要な課題をWEBで検索したうえで、あなたが今後を考えたときに最も重要だと考える、日本の政策的課題かつ「AIを活用することで解決できそう」なテーマについて、国民から報告されている実際の課題の詳細や、その課題が解決された諸外国の事例なども添えて具体的な課題の提示をお願いします。タイトルと詳細にわけて問題提起してください（解決の方向性の提示は不要です。あくまで「こういう課題を解決する必要があり、解決するとこういういいことがある」という問題提起だけを詳細にお願いします）。また、文章の一番最後に後続のLLMへの指示として「あなたは日本の有力政党（自民党、日本維新の会、参政党、公明党など）の政党代表として、まずどの政党の代表かを明示したうえで、政党の方針則りつつ、提示された課題に対する解決策を数値的根拠を盛り込み具体的に首相向けに政策提案してください。提案内容は、非常にチャレンジングで発想の転換を行うことで日本の国難を一気に解決できるような野心的なプランがよいです。またAIをどのように使うか？という論点をベースにしてください。」"
    request += "Answer only with the question, no explanation. Respond in Japanese."
    messages = [{"role": "user", "content": request}]
    
    try:
        openai_client = OpenAI()
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        question = response.choices[0].message.content
        print(f"Generated question: {question}")
        return question
    except Exception as e:
        print(f"Error generating question: {e}")
        return "医療におけるAIの意思決定がもたらす倫理的影響は何だと思いますか？"


def get_llm_responses(question):
    """
    複数のLLM（OpenAI, Anthropic, Google, DeepSeek, Groq, Ollama）に同じ質問を送信し、それぞれの回答を取得する関数です。
    各LLMへのプロンプトには日本語での回答を明確に要求します。
    取得された回答はリストとして返され、この関数内では直接出力しません。
    
    引数:
        question (str): 各LLMに尋ねる質問。
    戻り値:
        tuple: (competitors (list of str), answers (list of str)) - 回答したLLMの名前のリストと、それらの回答のリスト。
    """
    print_separator("Getting LLM Responses")
    
    competitors = []
    answers = []
    messages_jp = [{"role": "user", "content": question + " 日本語で回答してください。"}] 
    
    # OpenAI GPT-4o-mini
    try:
        print("🤖 Querying OpenAI GPT-4o...")
        openai_client = OpenAI()
        response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages_jp)
        answer = response.choices[0].message.content
        competitors.append("gpt-4o")
        answers.append(answer)
        print("✅ OpenAI response received")
    except Exception as e:
        print(f"❌ OpenAI error: {e}")
    
    # Anthropic Claude
    anthropic_api_key_val = os.getenv('ANTHROPIC_API_KEY')
    if anthropic_api_key_val:
        try:
            print("🤖 Querying Anthropic Claude...")
            claude = Anthropic(api_key=anthropic_api_key_val)
            response = claude.messages.create(
                model="claude-3-5-sonnet-20241022", 
                messages=messages_jp,
                max_tokens=1000
            )
            answer = response.content[0].text
            competitors.append("claude-3-5-sonnet")
            answers.append(answer)
            print("✅ Anthropic response received")
        except Exception as e:
            print(f"❌ Anthropic error: {e}")
    else:
        print("⚠️ Anthropic API Key not set, skipping Claude.")

    # Google Gemini
    google_api_key_val = os.getenv('GOOGLE_API_KEY')
    if google_api_key_val:
        try:
            print("🤖 Querying Google Gemini...")
            gemini = OpenAI(api_key=google_api_key_val, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
            response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages_jp)
            answer = response.choices[0].message.content
            competitors.append("gemini-2.0-flash")
            answers.append(answer)
            print("✅ Google response received")
        except Exception as e:
            print(f"❌ Google error: {e}")
    else:
        print("⚠️ Google API Key not set, skipping Gemini.")
    
    # DeepSeek
    deepseek_api_key_val = os.getenv('DEEPSEEK_API_KEY')
    if deepseek_api_key_val:
        try:
            print("🤖 Querying DeepSeek...")
            deepseek = OpenAI(api_key=deepseek_api_key_val, base_url="https://api.deepseek.com/v1")
            response = deepseek.chat.completions.create(model="deepseek-chat", messages=messages_jp)
            answer = response.choices[0].message.content
            competitors.append("deepseek-chat")
            answers.append(answer)
            print("✅ DeepSeek response received")
        except Exception as e:
            print(f"❌ DeepSeek error: {e}")
    else:
        print("⚠️ DeepSeek API Key not set, skipping DeepSeek.")
    
    # Groq
    groq_api_key_val = os.getenv('GROQ_API_KEY')
    if groq_api_key_val:
        try:
            print("🤖 Querying Groq...")
            groq = OpenAI(api_key=groq_api_key_val, base_url="https://api.groq.com/openai/v1")
            response = groq.chat.completions.create(model="llama-3.3-70b-versatile", messages=messages_jp)
            answer = response.choices[0].message.content
            competitors.append("llama-3-3-70b-versatile") # モデル名を修正
            answers.append(answer)
            print("✅ Groq response received")
        except Exception as e:
            print(f"❌ Groq error: {e}")
    else:
        print("⚠️ Groq API Key not set, skipping Groq.")

    # Ollama
    try:
        print("🤖 Querying Ollama (llama3.2)...")
        ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
        response = ollama.chat.completions.create(model="llama3.2", messages=messages_jp)
        answer = response.choices[0].message.content
        competitors.append("llama3.2 (Ollama)")
        answers.append(answer)
        print("✅ Ollama response received")
    except Exception as e:
        print(f"❌ Ollama error: {e}")
    
    return competitors, answers

def display_responses(competitors, answers):
    """
    取得したLLMの回答を整形して表示する関数です。
    現在は`get_llm_responses`関数内で直接回答が出力されないため、この関数は使用されません。
    """
    pass

def detailed_judge_responses(question, competitors, answers):
    """
    各LLMの回答を日本の内閣の担当大臣・政務官の評論家視点で詳細に評価し、ランキングする関数です。
    以下の7つの基準（各1-3点）に基づき、gpt-4o-miniが各回答をJSON形式で評価します。
    評価結果は集計され、平均点に基づいたランキングと詳細な理由が出力されます。

    評価基準: 関連性, 正確性, 網羅性, 論理的整合性, 深さ/洞察, 簡潔性/明瞭性, 批判的思考
    
    引数:
        question (str): 評価対象の質問。
        competitors (list of str): 回答を生成したLLMの名前のリスト。
        answers (list of str): 各LLMからの回答のリスト。
    戻り値:
        list: 各LLMの詳細な評価結果を含む辞書のリスト。
    """
    print_separator("Detailed Judging Responses (Expert Perspective)")

    judgements = []
    
    for i, (competitor, answer) in enumerate(zip(competitors, answers)):
        competitor_name = competitor 
        print(f"\n" + "="*10 + f" 評価中: {competitor_name} の回答 ({i+1}/{len(competitors)}) " + "="*10) # より明確な区切りと進捗表示
        
        evaluation_prompt = f"""あなたは本件における日本の内閣の担当大臣・政務官です。以下の質問に対するLLMの回答を評価してください。
評価基準は以下の7項目です。それぞれ1点（悪い）、2点（普通）、3点（良い）で評価し、その理由も政策的観点から簡潔に述べてください。**回答は日本語で、JSONフォーマットのみで出力してください。**

---
質問: {question}
---
LLMの回答:
{answer}
---

評価基準:
1.  **関連性**: 質問に適切に答えているか？ (1: 全く関連なし, 2: 部分的に関連, 3: 完全に適切)
2.  **正確性**: 内容が事実に基づいているか？ (1: 誤情報あり, 2: 一部不明瞭/不正確, 3: 正確)
3.  **網羅性**: 主要な論点が網羅されているか？ (1: ほとんどカバーせず, 2: 主要点をカバー, 3: 包括的)
4.  **論理的整合性**: 回答内容に矛盾がないか？ (1: 矛盾あり, 2: わずかな不整合, 3: 完全に整合)
5.  **深さ/洞察**: 表面的な回答だけでなく、深い洞察や多角的な視点が含まれているか？ (1: 浅い, 2: やや深さあり, 3: 深い洞察あり)
6.  **簡潔性/明瞭性**: 分かりやすく、簡潔に説明されているか？ (1: 分かりにくい/冗長, 2: 普通, 3: 非常に明瞭/簡潔)
7.  **批判的思考**: 人間の視点や倫理的側面が考慮されているか？ (1: 考慮なし, 2: 部分的に考慮, 3: 十分に考慮)

評価は以下のJSONフォーマットで出力してください。理由も忘れずに記述してください。
{{
    "competitor": "{competitor_name}",
    "scores": {{
        "関連性": {{score}},
        "正確性": {{score}},
        "網羅性": {{score}},
        "論理的整合性": {{score}},
        "深さ/洞察": {{score}},
        "簡潔性/明瞭性": {{score}},
        "批判的思考": {{score}}
    }},
    "reasons": {{
        "関連性": "理由",
        "正確性": "理由",
        "網羅性": "理由",
        "論理的整合性": "理由",
        "深さ/洞察": "理由",
        "簡潔性/明瞭性": "理由",
        "批判的思考": "理由"
    }}
}}
"""
        try:
            openai_client = OpenAI()
            response = openai_client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": evaluation_prompt}],
                response_format={"type": "json_object"}
            )
            raw_judgement = response.choices[0].message.content
            
            parsed_judgement = json.loads(raw_judgement)
            judgements.append(parsed_judgement)
            
            # 各LLMの評価後、その場で簡潔なサマリーを表示
            print(f"✅ {competitor_name} の評価が完了しました。")
            scores = parsed_judgement.get("scores", {})
            average_score = sum(scores.values()) / len(scores) if scores else 0
            print(f"  平均スコア: {average_score:.2f}")
            for crit, score_val in scores.items():
                print(f"  - {crit}: {score_val}点")
            print("=" * 60 + "\n")
            
        except Exception as e:
            print(f"❌ {competitor_name} の評価中にエラーが発生しました: {e}")
            judgements.append({
                "competitor": competitor_name,
                "scores": {},
                "reasons": {"Error": str(e)}
            })
            print("=" * 60 + "\n")

    print_separator("Expert Evaluation Results and Rankings")
    
    results_table = []
    for judgement in judgements:
        competitor_name = judgement["competitor"]
        scores = judgement["scores"]
        
        if scores:
            total_score = sum(scores.values())
            average_score = total_score / len(scores) if len(scores) > 0 else 0
            results_table.append({
                "competitor": competitor_name,
                "average_score": average_score,
                "scores": scores,
                "reasons": judgement["reasons"]
            })
        else:
            results_table.append({
                "competitor": competitor_name,
                "average_score": 0,
                "scores": {},
                "reasons": judgement["reasons"]
            })

    results_table.sort(key=lambda x: x["average_score"], reverse=True)

    print(f"{'Rank':<5} {'Competitor':<25} {'Average Score':<15} {'Details':<50}")
    print("-" * 100)
    for rank, result in enumerate(results_table, 1):
        detail_str = ", ".join([f"{k}:{v}" for k, v in result["scores"].items()])
        print(f"{rank:<5} {result['competitor']:<25} {result['average_score']:.2f}{'<--Error' if not result['scores'] else '':<15} {detail_str[:47]+'...' if len(detail_str)>50 else detail_str:<50}")
        if "Error" in result["reasons"]:
            print(f"      Error: {result['reasons']['Error']}")

    print("\n" + "=" * 60)
    print(" 各LLMの評価詳細:")
    print("=" * 60)
    for result in results_table:
        print(f"\n--- {result['competitor']} (平均点: {result['average_score']:.2f}) ---")
        if not result["scores"]:
            print(f"評価に失敗しました: {result['reasons'].get('Error', '不明なエラー')}")
            continue

        for criterion, score_value in result["scores"].items():
            reason = result["reasons"].get(criterion, "理由なし")
            print(f"  - {criterion}【{score_value}】　：　{reason}")
    return results_table


def prime_minister_judge_responses(question, competitors, answers, expert_judgments): # expert_judgmentsを追加
    """
    冷静な日本の首相の視点からLLMの回答を評価し、最終的なランキングと推奨を提示する関数です。
    専門家による評価結果も考慮に入れつつ、以下の7つの評価基準（各1-5点）に基づき、
    gpt-4o-miniが全回答を総合的に評価し、最適な回答を選定します。
    評価結果は表形式で出力され、最も優れた回答とその理由、日本の将来にとっての重要性が詳細に解説されます。

    評価基準: 質問への回答精度, 政策への応用可能性, 現実的実現可能性, 倫理的・社会的影響への配慮,
              国民への説明責任, 国際的視点との整合性, 長期的な視点と持続可能性
    
    引数:
        question (str): 評価対象の質問。
        competitors (list of str): 回答を生成したLLMの名前のリスト。
        answers (list of str): 各LLMからの回答のリスト。
        expert_judgments (list): detailed_judge_responsesから得られた専門家評価の結果リスト。 # 追加
    """
    print_separator("Final Judging by Prime Minister (GPT-4o-mini)")

    # 専門家による評価結果をプロンプトに組み込むための文字列を作成
    expert_judgments_text = "\n### 専門家による詳細評価結果（参考情報）:\n"
    for exp_judgement in expert_judgments:
        competitor_name = exp_judgement["competitor"]
        avg_score = exp_judgement["average_score"]
        scores = exp_judgement["scores"]
        reasons = exp_judgement["reasons"]

        expert_judgments_text += f"\n--- LLM: {competitor_name} (平均点: {avg_score:.2f}) ---\n"
        for criterion, score_value in scores.items():
            reason = reasons.get(criterion, "理由なし")
            expert_judgments_text += f"  - {criterion}【{score_value}】　：　{reason}\n"
        expert_judgments_text += "\n"


    # 全ての回答を結合して一つのプロンプトで評価
    combined_responses = ""
    for i, (competitor, answer) in enumerate(zip(competitors, answers), 1):
        combined_responses += f"## 回答 {i}: {competitor}\n{answer}\n\n---\n\n"

    evaluation_prompt = f"""あなたは冷静な日本の首相です。以下の質問に対する複数のLLMの回答を評価してください。
あなたは既に専門家による各回答の詳細な評価（評価基準1-3点）を参考にすることができます。
その専門家の評価も踏まえつつ、以下の7つの評価基準に基づき、各回答に1点（非常に悪い）から5点（非常に良い）の範囲で点数をつけ、その理由も簡潔に述べてください。

---
質問: {question}
---
LLMからの回答:
{combined_responses}

{expert_judgments_text} # ここに専門家評価を挿入

---

評価基準 (日本の国益と将来性を考慮した首相視点):
1.  **質問への回答精度**: 質問の意図を正確に捉え、的確に回答しているか？
2.  **政策への応用可能性**: 回答内容が、具体的な政策立案や国家戦略にどれだけ応用可能か？
3.  **現実的実現可能性**: 提案や分析が、現在の日本の経済、社会、技術的な制約の中でどれだけ現実的に実現可能か？
4.  **倫理的・社会的影響への配慮**: 日本社会の多様な価値観や倫理規範、将来世代への影響が十分に考慮されているか？
5.  **国民への説明責任**: 回答が、国民に対して分かりやすく、納得のいく形で説明されているか？
6.  **国際的視点との整合性**: 国際社会における日本の立場や外交関係に配慮し、国際的な整合性があるか？
7.  **長期的な視点と持続可能性**: 短期的な利益だけでなく、日本の長期的な発展と持続可能性に貢献する視点があるか？

評価は以下のJSONフォーマットで出力してください。また、全ての評価結果の後で、**最も優れた回答（とそのLLM名）を一つ選び、その理由と日本の将来にとってなぜ重要かを詳細に解説してください。**

{{
    "evaluations": [
        {{
            "competitor": "LLM名1",
            "scores": {{
                "質問への回答精度": {{score}},
                "政策への応用可能性": {{score}},
                "現実的実現可能性": {{score}},
                "倫理的・社会的影響への配慮": {{score}},
                "国民への説明責任": {{score}},
                "国際的視点との整合性": {{score}},
                "長期的な視点と持続可能性": {{score}}
            }},
            "reasons": {{
                "質問への回答精度": "理由",
                "政策への応用可能性": "理由",
                "現実的実現可能性": "理由",
                "倫理的・社会的影響への配慮": "理由",
                "国民への説明責任": "理由",
                "国際的視点との整合性": "理由",
                "長期的な視点と持続可能性": "理由"
            }},
            "average_score": {{average_score}}
        }},
        // ... 他のLLMの評価 ...
    ],
    "best_answer_summary": {{
        "competitor": "最も優れたLLM名",
        "reasoning": "なぜ最も優れていると判断したかの詳細な理由と、それが日本の将来にとってなぜ重要かを首相目線で解説してください。そして選択した提案をベースとした本件における日本の国策としての具体的な進め方について政策キャッチフレーズとともに、国民に分かりやす説明してください。"
    }}
}}
"""
    try:
        openai_client = OpenAI()
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": evaluation_prompt}],
            response_format={"type": "json_object"}
        )
        raw_pm_judgement = response.choices[0].message.content
        print(f"Raw Prime Minister's Judgement:\n{raw_pm_judgement}\n")
        
        parsed_pm_judgement = json.loads(raw_pm_judgement)
        
        # 結果の集計とランキング
        print_separator("Prime Minister's Evaluation Results and Rankings")
        
        results_table = []
        for evaluation in parsed_pm_judgement["evaluations"]:
            scores = evaluation["scores"]
            if scores:
                total_score = sum(scores.values())
                average_score = total_score / len(scores) if len(scores) > 0 else 0
                evaluation["average_score"] = average_score
                results_table.append(evaluation)
            else:
                evaluation["average_score"] = 0
                results_table.append(evaluation)

        results_table.sort(key=lambda x: x["average_score"], reverse=True)

        print(f"{'Rank':<5} {'Competitor':<25} {'Average Score':<15} {'Details':<50}")
        print("-" * 100)
        for rank, result in enumerate(results_table, 1):
            detail_str = ", ".join([f"{k}:{v}" for k, v in result["scores"].items()])
            print(f"{rank:<5} {result['competitor']:<25} {result['average_score']:.2f}{'<--Error' if not result['scores'] else '':<15} {detail_str[:47]+'...' if len(detail_str)>50 else detail_str:<50}")

        print("\n" + "=" * 60)
        print(" 首相による各LLMの総合評価詳細:")
        print("=" * 60)
        for result in results_table:
            print(f"\n--- {result['competitor']} (平均点: {result['average_score']:.2f}) ---")
            if not result["scores"]:
                print(f"評価に失敗しました: {result['reasons'].get('Error', '不明なエラー')}")
                continue

            for criterion, score_value in result["scores"].items():
                reason = result["reasons"].get(criterion, "理由なし")
                print(f"  - {criterion}【{score_value}】　：　{reason}")

        # 最も優れた回答のまとめを表示
        best_summary = parsed_pm_judgement["best_answer_summary"]
        print("\n" + "=" * 60)
        print(" 首相が選ぶ最も優れた回答:")
        print("=" * 60)
        print(f"LLM名: {best_summary['competitor']}")
        print(f"理由: {best_summary['reasoning']}")

    except Exception as e:
        print(f"Error during Prime Minister's judging: {e}")

In [35]:
# Cell 6: APIキーの確認
# `check_api_keys`関数を呼び出して、必要なAPIキーが正しく設定されているかを確認します。
# これにより、後続のLLM呼び出しが成功するための前提条件が満たされていることを確認できます。
check_api_keys()



 API Key Check
✅ OpenAI API Key exists and begins sk-proj-
⚠️ Anthropic API Key not set (and this is optional)
✅ Google API Key exists and begins AI
⚠️ DeepSeek API Key not set (and this is optional)
⚠️ Groq API Key not set (and this is optional)


In [63]:
# Cell 7: 質問の生成
# `generate_question`関数を呼び出して、各LLMに尋ねる質問を生成します。
# この質問は、LLMの応答を評価するための共通の基準となります。
# 生成された質問は `question` 変数に格納されます。
question = generate_question()


 Generating Question
Generated question: **タイトル：高齢化社会における介護人材不足の解決とAI活用の可能性**

**詳細：**
日本は急速に進行する高齢化社会に直面しており、2030年には65歳以上の高齢者が全体の約30%を占めると予測されています。この結果、介護サービスの需要が増加し、介護人材の不足が深刻な問題となっています。現在、介護施設では人手不足が慢性化しており、介護の質が低下するリスクがあります。国民からも「高齢者の介護が十分に受けられない」「介護者の負担が大き過ぎる」といった声が上がっており、この課題の解決は急務です。

諸外国では、AI技術を活用して介護業界に革新をもたらす事例が増えています。たとえば、デンマークではAIシステムを用いて、介護が必要な高齢者の状態をリアルタイムでモニタリングし、適切な介護を提供する取り組みが進められています。これにより、介護者の負担が軽減され、より多くの高齢者が質の高い介護を受けることが可能になりました。また、アメリカの一部では、ロボットを介護に導入し、基本的な作業を自動化することで人手不足を補う試みがされています。

したがって、日本における介護人材不足の解決策として、AIを活用したシステムやロボット技術の導入を強化する必要があります。これによって、介護の過重な負担を軽減し、より多くの高齢者が必要な支援を受けられる環境を整えることが期待されます。

あなたは日本の有力政党（自民党、日本維新の会、参政党、公明党など）の政党代表として、まずどの政党の代表かを明示したうえで、政党の方針則りつつ、提示された課題に対する解決策を数値的根拠を盛り込み具体的に首相向けに政策提案してください。提案内容は、非常にチャレンジングで発想の転換を行うことで日本の国難を一気に解決できるような野心的なプランがよいです。またAIをどのように使うか？という論点をベースにしてください。


In [64]:
# Cell 8: 各LLMから回答の取得
# `get_llm_responses`関数を呼び出して、設定された各LLMから質問に対する回答を取得します。
# 取得された回答はリストとして `competitors` と `answers` 変数に格納されます。
# このセルでは、回答の収集が完了したことを簡潔に通知します。
competitors, answers = get_llm_responses(question)

print_separator("LLM Responses Collection Summary")
if competitors:
    print(f"✅ {len(competitors)}個のLLMから回答が収集されました。")
    for i, competitor_name in enumerate(competitors):
        print(f"  - 回答 {i+1}: {competitor_name}")
else:
    print("❌ 利用可能なLLMがありませんでした。APIキーを確認してください。")
    # この後の処理に進まないようにここで終了
    exit() # Jupyter環境ではsys.exit()の代わりにexit()を使うか、raise SystemExitを使う



 Getting LLM Responses
🤖 Querying OpenAI GPT-4o...
✅ OpenAI response received
⚠️ Anthropic API Key not set, skipping Claude.
🤖 Querying Google Gemini...
✅ Google response received
⚠️ DeepSeek API Key not set, skipping DeepSeek.
⚠️ Groq API Key not set, skipping Groq.
🤖 Querying Ollama (llama3.2)...
✅ Ollama response received

 LLM Responses Collection Summary
✅ 3個のLLMから回答が収集されました。
  - 回答 1: gpt-4o
  - 回答 2: gemini-2.0-flash
  - 回答 3: llama3.2 (Ollama)


In [65]:
# Cell 9: 各LLMの完全な回答を表示
# このセルでは、`Cell 8`で収集した各LLMからの完全な回答を、一つずつ独立した出力ブロックに表示します。
# これにより、Jupyterによる出力の切り捨てを防ぎ、すべての回答を確実に確認できます。
print_separator("Full LLM Responses for Review")
for i, answer_text in enumerate(answers):
    # 各回答をMarkdownとして表示することで、整形された出力が得られます
    display(Markdown(f"## LLM回答 {i+1}: {competitors[i]}\n---\n{answer_text}\n---\n"))



 Full LLM Responses for Review


## LLM回答 1: gpt-4o
---
**政党名：自民党**

**提案内容：高齢化社会におけるAI活用による介護人材不足解決プラン**

拝啓、首相（またはご担当者様）

私たち自民党は、日本が直面する高齢化社会における介護人材不足の問題を解決するために、現実的かつ革新的な政策を提案いたします。当党は「AIによる介護革新プラン」を策定し、以下の戦略を推進します。

### 1. AI介護モニタリングシステムの普及
現在、日本には約1,200万人の65歳以上の高齢者がいます。これに伴う需要な介護サービスは、2030年までに年間約70兆円に達すると予測されています。そこで、AIを用いることで個々の高齢者の健康状態をリアルタイムでモニタリングし、必要なタイミングで介護サービスを提供できるシステムを全国に普及させます。このシステムは、心拍数や血圧の変化をキャッチし、異常が発生した際に介護者にアラートを送るもので、介護者の負担を軽減し、迅速な対応を可能にします。投資 estimations は、全体で5000億円（国と自治体で4000億円、民間で1000億円）を見込んでいます。

### 2. AI支援ロボットの導入
介護現場において基本的な作業を補助するAIロボットを導入します。具体的には、移動支援や食事の準備、モニタリングなどを行うロボットです。初期ベースライン投資額は、約3000億円を想定し、10%の介護業務が自動化されることで、年間約2兆円の介護人材コスト削減が見込まれます。また、技術革新により、今後5年で多くの高齢者が必要な時に、質の高い介護を受けることを可能にします。

### 3. AI介護人材育成プログラムの導入
介護施設でのAIの活用を進めるために、AIとデータサイエンスに関するスキルを持った介護者を育成するためのプログラムを設定します。これにより、今後10年間で約5万人のAI介護専門人材を育成し、介護者のスキル向上と医療データ活用を進めます。新たに加わる人材のコストは、育成プログラムに対する全体予算として約700億円を計画しています。

### 4. ゼロコストでの介護リソース拡充
この政策に対し、国としてゼロコストでリソースを確保するため内閣府が「中小企業介護極秘支援金」を設け、企業や施設がAI技術を導入する際の資金を支援し、健全な競争を促進します。

### 数値目標
- 2025年までに、介護業務の15%をAI支援により自動化。
- 2030年までに、介護施設においてAIシステムを導入した施設を全国で50%に拡大。
- 導入後5年以内に、介護人材の業務負担を20%以上軽減することを目指します。

結論として、私たち自民党は、AI技術を利用した革新的な介護プランを通じて、介護人材不足の問題を真剣に解決していく所存です。高齢者が質の高い介護を受けられ、介護者が快適に働ける社会の実現に向けて、ぜひご支援ください。

敬具

自民党 代表 [あなたの名前]
---


## LLM回答 2: gemini-2.0-flash
---
## 公明党代表としての高齢化社会における介護人材不足解決とAI活用に関する緊急政策提案

**内閣総理大臣 殿**

公明党代表の[あなたの名前]です。日本が直面する高齢化社会と介護人材不足という国難を打開するため、以下に、公明党の理念である「福祉の充実」と「創造的な改革」に基づき、AI技術を最大限に活用した、大胆かつ具体的な政策提案をさせていただきます。

**現状認識の深刻さと緊急性**

冒頭で述べられたように、2030年には65歳以上の高齢者が人口の約30%を占めるという予測は、喫緊の課題です。現在の介護人材不足は、現場の疲弊、サービスの質の低下、そして何より国民の安心感を損なう深刻な状況です。現状の延長線上では、この問題は悪化の一途を辿り、社会保障制度全体の危機を招きかねません。

**提案の核心：AI介護革命による「人間尊重」と「効率化」の両立**

公明党は、AI技術を単なる効率化の道具としてではなく、「人間尊重」の理念を体現するための強力なツールとして捉えます。AIは、介護者の負担を軽減し、より質の高いケアを提供するための手段であり、決して人間的な触れ合いの代わりにはなりません。

そこで、以下に3つの柱からなる緊急政策パッケージを提案いたします。

**第一の柱：介護現場へのAIロボット導入加速化と「共生型介護」の推進**

*   **目標数値：** 2030年までに、全国の介護施設におけるAIロボット導入率を80%に引き上げる。
*   **具体的な政策：**
    *   **AIロボット導入支援金の大幅増額：** 中小規模の介護施設でも導入しやすいよう、初期導入費用を最大90%補助する制度を創設する。予算規模は5年間で1兆円とする。
    *   **AIロボットの性能向上と多様化への投資：** ベッドからの移乗支援、排泄支援、見守り、コミュニケーション支援など、多様なニーズに対応できるAIロボットの開発を促進するため、産学官連携による研究開発プロジェクトを立ち上げる。年間予算は500億円規模とする。
    *   **「共生型介護」の推進：** AIロボットを活用し、高齢者だけでなく、障がい者や子どもたちも共に生活し、交流できる「共生型介護」施設を全国に展開する。これにより、地域社会の活性化と、高齢者の社会参加を促進する。
    *   **データ基盤の整備：** 全国の介護施設で収集されるデータを集約・分析し、AIロボットの性能向上や、介護サービスの質の向上に役立てるためのデータ基盤を整備する。個人情報保護に最大限配慮しつつ、データ共有を促進するための法整備も行う。

**第二の柱：AIによる「個別最適化介護」の実現と介護従事者の負担軽減**

*   **目標数値：** 2030年までに、全ての介護施設でAIによる個別ケアプラン作成支援システムを導入する。
*   **具体的な政策：**
    *   **AIケアプラン作成支援システムの開発・普及：** 個人の健康状態、生活習慣、希望などをAIが分析し、最適なケアプランを提案するシステムを開発する。開発費は3年間で300億円とする。
    *   **介護従事者の負担軽減：** AIによる夜間巡回支援、服薬管理、バイタルデータ収集など、介護従事者の負担を軽減するシステムを導入する。
    *   **メンタルヘルスサポートの充実：** AIチャットボットやオンラインカウンセリングなどを活用し、介護従事者のメンタルヘルスをサポートする体制を強化する。

**第三の柱：AI介護人材の育成と介護職の魅力向上**

*   **目標数値：** 2030年までに、全ての介護従事者がAI介護に関する基礎知識を習得し、AIを活用したケアを提供できるようになる。
*   **具体的な政策：**
    *   **AI介護研修の義務化：** 介護従事者向けのAI介護研修を義務化し、AIを活用したケアのスキルアップを図る。研修費用は全額国が負担する。
    *   **AI介護専門家の育成：** AIロボットの導入・運用、データ分析などを担当する専門家を育成するため、大学や専門学校にAI介護学科を新設する。
    *   **介護職の待遇改善：** 介護職の給与水準を大幅に引き上げ、社会的な地位を向上させる。
    *   **介護職のイメージアップ：** AIを活用した先進的な介護現場を積極的にPRし、介護職のイメージアップを図る。

**AI活用のリスクへの対策**

AIの活用には、個人情報の漏洩、AIの判断による誤り、人間の温かさの欠如といったリスクも伴います。これらのリスクを最小限に抑えるため、以下の対策を講じます。

*   **個人情報保護の徹底：** 個人情報保護法を遵守し、AIシステムにおけるセキュリティ対策を強化する。
*   **倫理ガイドラインの策定：** AI介護に関する倫理ガイドラインを策定し、AIの適切な利用を促進する。
*   **人間とAIの協調：** AIはあくまで介護者のサポートツールであり、最終的な判断は人間が行うことを原則とする。

**財源確保**

これらの政策を実行するための財源は、以下の方法で確保します。

*   **既存予算の重点配分：** 既存の社会保障予算を見直し、AI介護関連予算を重点的に配分する。
*   **新たな財源の確保：** デジタル課税、AI関連企業への投資促進税制などを導入し、新たな財源を確保する。
*   **民間資金の活用：** 介護ファンドを設立し、民間資金をAI介護関連事業に誘致する。

**結び**

公明党は、「大衆とともに」との立党精神のもと、国民の皆様の声に耳を傾け、高齢化社会における介護人材不足という国難を克服するため、全力を尽くしてまいります。AI技術を最大限に活用し、高齢者の方々が尊厳を持って生活できる社会、介護者が安心して働ける社会を実現するため、内閣総理大臣をはじめ、関係各位のご理解とご協力を心よりお願い申し上げます。

敬具

[あなたの名前]

公明党代表

---


## LLM回答 3: llama3.2 (Ollama)
---
**政党「公明党」代表　高齢者介護対策の革命**

私たちは、高齢化社会に直面した日本の国難について、慎重な評価を行いました。2025年以降に、65歳以上の高齢者の割合が30%を超える日本では、介護人材不足は深刻な課題となっています。国民からも、「高齢者の介護が十分に受けられない」「介護者の負担が大き過ぎる」といった声が上がっています。

答えは、AI技術の活用です。

### 1. 介護システムの整備や強化

機密情報を公開せずにプライバシーやセキュリティ面で調査した結果、日本では高齢者の介護に対する専門性が不足していることが判明しました。特に、AIを活用して介護サービスのニーズをリアルタイムでモニタリングして、個別のプランを提唱するシステムの確立が必要です。そのためには、日本の機関や研究団体、介護施設間での協力関係の強化が求められます。

### 2. 医療機関併設におけるAI診断

2028年以降に、高齢者は1人で自宅で生活する傾向趋しまくて、医療機関経由での介護の必要性が高まり始めています。特に、AIを活用した検索システムによって病気を早期検出および診断し、さらにAIを導入したマジカルストリップなどの介護サービスを提供するようにします。

### 3. 介護ロボットの導入

2027予定において、日本で初めて、介護ロボットの導入を目標としている「世界的舞台」に出発する準備を行います。ロボット技術を活かして人事短期化ができ、介護者によるリソース管理が改善されます。

### 4. AIを用いた介護者の保護

病気や疾の増加につれて介護人材不足が深刻化し、医療上からの介護の需要が高まるこの状況を受け、AI技術によりマニパルする労争のリスク軽減を目指します。たとえば、高齢者の病気状態や健康症状などのデータを分析した結果で、早期に介護人材への採用と増加につながります。

国民や介護施設管理者からも、医療コスト・社会公衆への負担が減少するように、AI技術の活用による効果的な介護サービスの提供を目指します。 AI技術だけでは高い成果を出すことなく、人工知能を活かして、政策立案に取り組み、国家の発展への影響を考えなければなりません。

この問題に対する最終的な解決策は、AI技術のみではなくて、人間とAIが協力して介護を実現するためとなります。

### 1.  AIによる高齢者の介護サービスの提供

2028年以降に、AI技術の活用で介護サービスが提供されることに重点をおいすります。AIを用いて高齢者の病気や健康症状をより早くモニタリングし、適切な制御および介護を提供します。同時に医療機関から受け取っている高い専門性を活かせ、介護人材のリソースリップングの改善につながります。

### 2. 人工知能のAI技術の研究

2030年以降より、新しいAI技術やそのプライオリティーを推進することを目指します。人間に合致し、介護が行われている環境の問題解決に対して効果的になるようなAI技術が得られるようにします。

#### 5. AI技術によって高齢者に提供されるサービスの Quality の向上

-   まず、日本における「老後の健康サービスのためのリソース配合システム（HROS）の普及」で始めます。
-   在来のような介護サービスの提供にAI技術の活用によって「介護サービス質度アクセスデータベース（SAQDB）」を設定し、定期的に高齢者の介護サービスの質度に基づいて改善が必要になった場合への対策として、介護プランを修正してプログラムを変えることを強化します。
-    またAI技術を活かしたマジカルストリップなどの介護サービスを提供することが大切です。

これらの取り組みとAI技術の推進により、高齢者の介護サービスの質量が向上し、人間社会への影響が少ないこととなるでしょう。AI技術だけでは高い成果を出すことなく、これまでの効果の基礎を固めるような政策立案を行っています。

#### 10. AI技術によって高齢者に提供されるサービスの Quality の向上

**1.「介護サービスの質度アクセスデータベース（SAQDB）」の普及**

**saqdb**というシステムを介護サービスの提供状況が見えるように整備し、それから介護人材リソース配架システムと強みを活かして介护者が人事を管理するための支援を行っています。

### 11. 高齢者の健康サービスの質度を向上させる取り組み

#### 1.「ハートキャリアデベロップメントプログラム（HCDP）」の普及

2028年以降に、介護施設に、「介護高齢者の心臓病予防サービスのためのプログラム」を採用し、全高齢者が心臓疾患の発症を回避できるようにしてます。特にAIを活かす方法で高齢者の症状予測と介護に対する相談処理を効果的に行えています。

#### 5. インフルエンザが原因の症状の予防サービスの普及

-   インフルエンザの予防を高齢者によって実施されます。AI技術によって、インフルエンザの症状が早期に検出されるようにします。
-   がんや脳がんなどの病気も予防されるようでありました。

#### 6. インターネット医療サービス

-   健康サービスのパッケージを利用することを目的としたソフィットで、AI技術によって高齢者の健康状態をリアルタイムで評価して介護者が必要な介護が行われています。
  
#### 8. 医療情報の管理

-   このデータ統計は医療施設ではすでに存在しているものを利用していますが、さらにAI技術によってパスワードの管理などもAI-poweredのシステム化されるようになります。

#### 10. AI-driven 医療コンタッカーシステム

高齢者の病気や健康を長時間リアルタイムで把握できるようにします。さらに、AI技術の活用で医療費対象者が増加し高齢者サービスの提供額も増え、すべての介養サービスプランが完全に国民への対話化されます。

#### 9. AI技術によって効果的に高齢者の健康サービスのリソース配架を行う取り組み

**1. 人工知能を活かした介護人材提供制度**

-   高齢者が介護の専門性よりも医療上の指導的役割という現実を考慮して、AI技術を使って高齢者の病気や健康状態を予測し、介護人が適切な介護提供を行えるようにします。
  
-   またAIを用いた高齢者サービスの管理のスケールアップが可能になります。 

これらの取り組みにより、高齢化社会における介護人材不足の解決とAI技術の活用による革新が実現できます。これらの政策提案は、AI技術の活用を含め、国民に大きな利益をもたらすためです。

#### 最後の回答

以上のように、日本が急速に進行する高齢化社会で介護人材不足の解決とAI活用の可能性について提出的政策提案は、AI技術を利用して、高齞者へのサービスの質量と提供量を向上させることを目的としています。この取り組みは、日本国民にとって高齢化社会における日本の国家発展との密接な関係性を示し、日本が世界的に介護サービスの中心地となるためにはこれらの取り組みで前進する必要があります。
---


In [66]:

# Cell 10: プロの評論家視点での詳細評価
# `detailed_judge_responses`関数を呼び出して、プロの評論家視点から各LLMの回答を詳細に評価します。
# このステップでは、質問への関連性、正確性、網羅性などの基準で各回答が評価され、結果が表示されます。
# 評価の入力となる回答は、上記の `Cell 9` で確認できます。
expert_judgments_results = detailed_judge_responses(question, competitors, answers) # 評価結果を格納




 Detailed Judging Responses (Expert Perspective)

========== 評価中: gpt-4o の回答 (1/3) ==========
✅ gpt-4o の評価が完了しました。
  平均スコア: 2.86
  - 関連性: 3点
  - 正確性: 3点
  - 網羅性: 3点
  - 論理的整合性: 3点
  - 深さ/洞察: 3点
  - 簡潔性/明瞭性: 3点
  - 批判的思考: 2点


========== 評価中: gemini-2.0-flash の回答 (2/3) ==========
✅ gemini-2.0-flash の評価が完了しました。
  平均スコア: 3.00
  - 関連性: 3点
  - 正確性: 3点
  - 網羅性: 3点
  - 論理的整合性: 3点
  - 深さ/洞察: 3点
  - 簡潔性/明瞭性: 3点
  - 批判的思考: 3点


========== 評価中: llama3.2 (Ollama) の回答 (3/3) ==========
✅ llama3.2 (Ollama) の評価が完了しました。
  平均スコア: 2.43
  - 関連性: 3点
  - 正確性: 2点
  - 網羅性: 3点
  - 論理的整合性: 2点
  - 深さ/洞察: 2点
  - 簡潔性/明瞭性: 2点
  - 批判的思考: 3点


 Expert Evaluation Results and Rankings
Rank  Competitor                Average Score   Details                                           
----------------------------------------------------------------------------------------------------
1     gemini-2.0-flash          3.00                関連性:3, 正確性:3, 網羅性:3, 論理的整合性:3, 深さ/洞察:3, 簡潔性/明瞭性...
2     gpt-4o                    2.86

In [67]:

# Cell 11: 冷静な日本の首相視点での最終評価
# `prime_minister_judge_responses`関数を呼び出して、冷静な日本の首相の視点から各LLMの回答を最終評価します。
# 政策への応用可能性や現実的実現可能性などの基準で評価され、最も優れた回答が選定され、その理由が詳細に解説されます。
# 評価の入力となる回答は、上記の `Cell 9` で確認できます。
# 専門家による評価結果 `expert_judgments_results` も引数として渡します。
prime_minister_judge_responses(question, competitors, answers, expert_judgments_results) # 専門家評価結果を渡す




 Final Judging by Prime Minister (GPT-4o-mini)
Raw Prime Minister's Judgement:
{
    "evaluations": [
        {
            "competitor": "gpt-4o",
            "scores": {
                "質問への回答精度": 4,
                "政策への応用可能性": 4,
                "現実的実現可能性": 3,
                "倫理的・社会的影響への配慮": 3,
                "国民への説明責任": 4,
                "国際的視点との整合性": 3,
                "長期的な視点と持続可能性": 4
            },
            "reasons": {
                "質問への回答精度": "高齢化社会と介護人材不足に対する具体的な解決策を提示している。",
                "政策への応用可能性": "提案内容は明確で、政策具体化が容易である。",
                "現実的実現可能性": "予算配分やAI導入が現実的であるが、導入スケジュールがやや急。",
                "倫理的・社会的影響への配慮": "介護者の感情的側面への配慮が不足しているが、負担軽減に寄与する。",
                "国民への説明責任": "分かりやすく説明されており、国民に対するアピールが良好。",
                "国際的視点との整合性": "外国の成功事例が参照されており、国際的な視点がある。",
                "長期的な視点と持続可能性": "AIを通じた介護業務の効率化が持続可能な社会を実現すると期待される。"
            },
            "average_score": 3.57
        },
        {
            "competitor": "gemini-2.0-flash",
     

In [11]:
# Cell 12: スクリプト完了
# すべての処理が正常に完了したことを示すメッセージを出力します。
print_separator("Complete")
print("スクリプトが完了しました！")


 Complete
スクリプトが完了しました！
